In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
import random
from pathlib import Path

import pandas as pd
import torch
from torch.nn.functional import elu
import numpy as np
import matplotlib.pyplot as plt

from alpaca.uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from alpaca.model.ensemble import MLPEnsemble
from alpaca.uncertainty_estimator import build_estimator
from alpaca.analysis.metrics import get_uq_metrics

plt.rcParams['figure.facecolor'] = 'white'


In [ ]:
SEED = 10
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.cuda.set_device(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
folder = Path('./data/regression')
files = [file for file in os.listdir(folder) if file.endswith('.pickle')]
# files = [file for file in files if file.startswith('kin8')]


In [ ]:

accumulate = []

In [ ]:
data = []
for cnt, file in enumerate(files):
    print(file)
    with open(folder / 'log_exp.log', 'w') as f:
        f.write(f'{cnt} / {len(files)}')
    with open(folder / file, 'rb') as f:
        dct = pickle.load(f)
    print(file)
    config = dct['config']
    config['n_ue_runs'] = 1
    config['acc_percentile'] = .1
    state_dict = dct['state_dict']
    x_train, y_train, x_val, y_val, x_scaler, y_scaler = dct['data']

    ensemble = MLPEnsemble(
        config['layers'], n_models=config['n_ens'], activation = elu,
        reduction='mean')
    ensemble.load_state_dict(state_dict)

    model = ensemble.models[2]

    x_val_tensor = torch.tensor(x_val)
    predictions = model(x_val_tensor.cuda()).cpu().detach().numpy()
    unscale = lambda y : y_scaler.inverse_transform(y)
    # unscale = lambda y : y

    scaled_errors = unscale(predictions) - unscale(y_val)
    rmse_single = np.sqrt(np.mean(np.square(scaled_errors)))
    #%%
    predictions = ensemble(x_val_tensor.cuda()).cpu().detach().numpy()
    scaled_errors = unscale(predictions) - unscale(y_val)
    rmse_ensemble = np.sqrt(np.mean(np.square(scaled_errors)))

    #%%
    accumulate.append([file[:4], 'single', rmse_single])
    accumulate.append([file[:4], 'ensemble', rmse_ensemble])

    print(file)
    print('Single', rmse_single)
    print('Ensemble', rmse_ensemble)



In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
df = pd.DataFrame(accumulate, columns=['dataset', 'type', 'rmse'])
df

In [ ]:
sns.boxplot('dataset', 'rmse', hue='type', data=df)
plt.savefig('rmse.png', dpi=150)
